In [3]:
import pandas as pd
import os
import glob

#non full path is being funky
print(os.getcwd())

/home/jovyan/596nfl/Data


In [49]:
#Reading in all data
#tracking data 2 and 5 have something wonky happening with the file formatting, which is fixed by ignoring 'bad' lines and using python engine.

games = pd.read_csv('/home/jovyan/596nfl/Data/games.csv')
plays = pd.read_csv('/home/jovyan/596nfl/Data/plays.csv')
players = pd.read_csv('/home/jovyan/596nfl/Data/players.csv')
player_play = pd.read_csv('/home/jovyan/596nfl/Data/player_play.csv', low_memory=False)

# Gets a list of all tracking_data files
file_list = glob.glob('/home/jovyan/596nfl/Data/tracking_week_*.csv')
dataframes = []
trackSize = 0

for file in file_list:
    df = pd.read_csv(file, engine='python', on_bad_lines='skip')
    filtered_df = df[df['event'].notna()]   # Filter out rows where 'event' is not NA
    dataframes.append(filtered_df)
    trackSize += df.shape[0]

# Merge all the DataFrames into one
merged_tracking_data = pd.concat(dataframes, ignore_index=True)

# Save the merged DataFrame to a new file
merged_tracking_data.to_csv('merged_tracking_data.csv', index=False)



In [50]:
#(from tracking) roughly 96% useless data removal yay
print(trackSize)
print(merged_tracking_data.shape[0])
print(1 - merged_tracking_data.shape[0]/trackSize)

1917409
81734
0.9573726836579989


In [65]:
reduced_merged_tracking_data = merged_tracking_data[merged_tracking_data['frameType'] != "AFTER_SNAP"]   # Filter out rows where 'frameType' is AFTER_SNAP
print(reduced_merged_tracking_data.shape[0])

# These players? have no nflId when club == football, and while this does remove some it doesn't for other cells so I am confused
reduced_merged_tracking_data = reduced_merged_tracking_data[reduced_merged_tracking_data['club'] != "football"]          
print(reduced_merged_tracking_data.shape[0])



#we don't need these i think
columns_to_delete = ['s', 'a', 'jerseyNumber', 'displayName'] 

# Drop the columns
reduced_merged_tracking_data = reduced_merged_tracking_data.drop(columns=columns_to_delete)




# Save the updated DataFrame back to a CSV if needed
reduced_merged_tracking_data.to_csv('reduced_merged_tracking_data.csv', index=False)



# Get the value at row 67 and column 'club'
# reduced_merged_tracking_data.loc[67, 'club']

38181
36532


In [52]:



# hi















In [42]:
# Merge the datasets on gameId
merged_data = pd.merge(plays, games, on='gameId', how='inner')

# Merge the tables on gameId and playId
secondary_merged_data = pd.merge(merged_data, player_play, on=['gameId', 'playId'], how = 'inner')

# Check the shape of the merged data to ensure it's correct
print(f"Shape of merged data: {secondary_merged_data.shape}")

tert_merged_data = pd.merge(secondary_merged_data, players[['nflId', 'displayName', 'position']], on='nflId', how='inner')

print(f"Shape of merged data: {tert_merged_data.shape}")

Shape of merged data: (183802, 106)
Shape of merged data: (183802, 108)


In [45]:
# Filter out rows where wasInitialPassRusher and defensiveCoverageAssignment are both NA
filtered_data = tert_merged_data[
    ~(tert_merged_data['wasInitialPassRusher'].isna() & tert_merged_data['pff_defensiveCoverageAssignment'].isna())
]

# Check the shape of the filtered data to ensure correctness
print(f"Shape of data after filtering: {filtered_data.shape}")



Shape of data after filtering: (56126, 108)
